In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Define the number of modes and time steps
n = 5  # Number of modes
T = 1000  # Number of time steps

In [ ]:
# Example input signal (n x T) with complex values
input_signal = np.zeros((n, T), dtype=complex)
input_signal[0, :] = 0.0 + 0*1j
input_signal[1, :] = 1.0 + 0*1j
input_signal[2, :] = 0.0 + 0*1j
input_signal[3, :] = 0.0 + 0*1j
input_signal[4, :] = 0.0 + 0*1j


In [ ]:
# Define noise parameters
mean_real = 0
std_real = np.sqrt(0.25)
mean_imag = 0
std_imag = np.sqrt(0.25)

# Generate Gaussian noise for real and imaginary parts
noise_real = np.random.normal(mean_real, std_real, size=(n, T))
noise_imag = np.random.normal(mean_imag, std_imag, size=(n, T))

# Add noise to the input signal
noisy_input_signal = np.real(input_signal) + noise_real + 1j * (np.imag(input_signal) + noise_imag)

In [ ]:
output_signal = noisy_input_signal

real_quadratures = np.real(output_signal)
imag_quadratures = np.imag(output_signal)

# Stack real and imaginary parts
combined_quadratures = np.vstack((real_quadratures, imag_quadratures))

# Estimate the full covariance matrix
full_covariance = np.cov(combined_quadratures)

cov = np.cov(combined_quadratures)

In [ ]:
cov

In [ ]:
# Let's do this many times
def calculate_cov():
    # Define the number of modes and time steps
    n = 5  # Number of modes
    T = 1000  # Number of time steps

    # Example input signal (n x T) with complex values
    input_signal = np.zeros((n, T), dtype=complex)
    input_signal[0, :] = 0.0 + 0*1j
    input_signal[1, :] = 1.0 + 0*1j
    input_signal[2, :] = 0.0 + 0*1j
    input_signal[3, :] = 0.0 + 0*1j
    input_signal[4, :] = 0.0 + 0*1j

    # Define noise parameters
    mean_real = 0
    std_real = np.sqrt(0.25)
    mean_imag = 0
    std_imag = np.sqrt(0.25)

    # Generate Gaussian noise for real and imaginary parts
    noise_real = np.random.normal(mean_real, std_real, size=(n, T))
    noise_imag = np.random.normal(mean_imag, std_imag, size=(n, T))

    # Add noise to the input signal
    noisy_input_signal = np.real(input_signal) + noise_real + 1j * (np.imag(input_signal) + noise_imag)

    output_signal = noisy_input_signal

    real_quadratures = np.real(output_signal)
    imag_quadratures = np.imag(output_signal)

    # Stack real and imaginary parts
    combined_quadratures = np.vstack((real_quadratures, imag_quadratures))

    # Estimate the full covariance matrix
    full_covariance = np.cov(combined_quadratures)

    cov = np.cov(combined_quadratures)

    return cov

In [ ]:
cov = []
for _ in range(10000):
    cov.append(calculate_cov())

avg_cov = np.mean(cov, axis=0)


In [ ]:
avg_cov

In [ ]:
# In order for this technique to be useful, let's modify the function
def calculate_cov(operator):
    # Define the number of modes and time steps
    n = 4  # Number of modes
    T = 1000  # Number of time steps

    # Example input signal (n x T) with complex values
    input_signal = np.zeros((n, T), dtype=complex)
    input_signal[0, :] = 0.0 + 0*1j
    input_signal[1, :] = 1.0 + 0*1j
    input_signal[2, :] = 0.0 + 0*1j
    input_signal[3, :] = 0.0 + 0*1j

    # Define noise parameters
    mean_real = 0
    std_real = np.sqrt(0.25)
    mean_imag = 0
    std_imag = np.sqrt(0.25)

    # Generate Gaussian noise for real and imaginary parts
    noise_real = np.random.normal(mean_real, std_real, size=(n, T))
    noise_imag = np.random.normal(mean_imag, std_imag, size=(n, T))

    # Add noise to the input signal
    noisy_input_signal = np.real(input_signal) + noise_real + 1j * (np.imag(input_signal) + noise_imag)

    output_signal = operator(noisy_input_signal)

    real_quadratures = np.real(output_signal)
    imag_quadratures = np.imag(output_signal)

    # Stack real and imaginary parts
    combined_quadratures = np.vstack((real_quadratures, imag_quadratures))

    # Estimate the full covariance matrix
    full_covariance = np.cov(combined_quadratures)

    cov = np.cov(combined_quadratures)

    return cov

In [ ]:
# Now `calculate_cov` applies a function to our signal to obtain the output signal. 
# in order to use the new function, we have to define the function that will allow us to obtain our output signal
from jax import config
config.update("jax_enable_x64", True)
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import sax
import pandas as pd

from simphony.quantum import QuantumTimeElement
from simphony.libraries import siepic, ideal
from simphony.utils import smooth_rectangular_pulse, dict_to_matrix

from simphony.baseband_vector_fitting import BasebandModel
from scipy.signal import dlsim

netlist = {
    "instances": {
        "wg": "waveguide",
        "hr": "half_ring",
    },
    "connections": {
        "hr,port 3": "wg,o0",
        "hr,port 2": "wg,o1",
    },
    "ports": {
        "o0": "hr,port 1",
        "o1": "hr,port 4",
    }
}

circuit, info = sax.circuit(
    netlist=netlist,
    models={
        "waveguide": siepic.waveguide,
        "half_ring": siepic.bidirectional_coupler,
    }
)

wvl_microns = np.linspace(1.51, 1.59, 200)
center_wvl = 1.55

ckt = circuit(wl=wvl_microns, wg={"length": 45, "loss": 100})
s_params = np.copy(np.asarray(dict_to_matrix(ckt)))


model = BasebandModel(wvl_microns, center_wvl, s_params, 50)
model.plot()



In [ ]:
sys = model.generate_sys_discrete()

In [ ]:
def mydlsim(system, u, t=None, x0=None):
    if t is None:
        out_samples = len(u)
        stoptime = (out_samples - 1) * system.dt
    else:
        stoptime = t[-1]
        out_samples = int(np.floor(stoptime / system.dt)) + 1

    # Pre-build output arrays
    xout = np.zeros((out_samples, system.A.shape[0]))
    yout = np.zeros((out_samples, system.C.shape[0]))
    tout = np.linspace(0.0, stoptime, num=out_samples)

    # Check initial condition
    if x0 is None:
        xout[0, :] = np.zeros((system.A.shape[1],))
    else:
        xout[0, :] = np.asarray(x0)

    # Pre-interpolate inputs into the desired time steps
    if t is None:
        u_dt = u
    else:
        if len(u.shape) == 1:
            u = u[:, np.newaxis]

        u_dt_interp = interp1d(t, u.transpose(), copy=False, bounds_error=True)
        u_dt = u_dt_interp(tout).transpose()

    # Simulate the system
    for i in range(0, out_samples - 1):
        xout[i+1, :] = (np.dot(system.A, xout[i, :]) +
                        np.dot(system.B, u_dt[i, :]))
        yout[i, :] = (np.dot(system.C, xout[i, :]) +
                      np.dot(system.D, u_dt[i, :]))

    # Last point
    yout[out_samples-1, :] = (np.dot(system.C, xout[out_samples-1, :]) +
                              np.dot(system.D, u_dt[out_samples-1, :]))

    return tout, yout

In [ ]:

n = 2  # Number of modes
T = 10000  # Number of time steps

# Example input signal (n x T) with complex values
# x0 = np.zeros((100), dtype=complex)
input_signal = np.zeros((n, T), dtype=complex)
input_signal[0, :] = 0.0 + 0*1j
input_signal[1, :] = 1.0 + 0*1j


def simulate(sys, u, t=None, x0=None):
    # Condition needed to ensure output remains compatible
    # is_ss_input = isinstance(system, StateSpace)

    # Check initial condition
    if x0 is None:
        xout = np.zeros((sys.A.shape[1],u.shape[1]))
    else:
        xout = np.asarray(x0)

    # Simulate the system
    xout = (np.dot(sys.A, xout) +
                        np.dot(sys.B, u))
    yout = (np.dot(sys.C, xout) +
                    1.0*np.dot(sys.D, u))

    return yout, xout

y, x = mydlsim(sys, input_signal)

# input_signal[:, 2] = 0.0 + 0*1j
# input_signal[:, 3] = 0.0 + 0*1j
# t, y, x = dlsim(sys, input_signal, x0 = x0)
# plt.plot(t, np.abs(y[:, 1])**2)

In [ ]:
plt.plot(np.abs(y[0, :])**2)

In [ ]:
y.shape